In [1]:
from tpot import TPOTClassifier
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import RepeatedKFold

In [2]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
sample = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv', index_col='id')
all_df = pd.concat([train, test], axis=0)

In [3]:
all_df = all_df.drop(['id', 'target'], axis=1)

In [4]:
lbe = LabelEncoder()
train['target'] = lbe.fit_transform(train['target'])

In [5]:
le = LabelEncoder()
for col in all_df.columns:
    all_df[col] = le.fit_transform(all_df[col])


In [6]:
for col in all_df.columns:
    all_df[col] = np.log1p(all_df[col])

In [7]:
train_df = all_df[:len(train)]
train_df['target'] = train['target']
test_df = all_df[len(train):]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Modeling

In [8]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)


In [9]:
pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=cv, random_state=42, verbosity=2, scoring = 'neg_log_loss', max_time_mins=240, early_stop=5)

In [10]:
pipeline_optimizer.fit(train_df.drop(['target'], axis=1), train_df['target'])

Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -1.1000373346055503

Generation 2 - Current best internal CV score: -1.1000373346055503

Generation 3 - Current best internal CV score: -1.1000373346055503

Generation 4 - Current best internal CV score: -1.1000373346055503

Generation 5 - Current best internal CV score: -1.1000373346055503

Best pipeline: BernoulliNB(input_matrix, alpha=10.0, fit_prior=True)


TPOTClassifier(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=1),
               early_stop=5, generations=5, max_time_mins=240,
               population_size=20, random_state=42, scoring='neg_log_loss',
               verbosity=2)

In [11]:
pipeline_optimizer.export('tpot_pipeline.py')

In [12]:
preds = pipiline_optimizer.predict_proba(test_df)

NameError: name 'pipiline_optimizer' is not defined

In [13]:
sample[sample.columns] = preds
sample.to_csv('tpot_optimized.csv')

NameError: name 'preds' is not defined